<a href="https://colab.research.google.com/github/swahareddy/gpt2AI_whatsapp_simulation/blob/master/Train_a_GPT_2_Text_Generating_Model_w_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whatsapp Chat simulation by OpenAI's GPT-2. 
Made easily accesible by https://github.com/minimaxir/gpt-2-simple 

In [3]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Link your Google drive storage and add the file you exported (`WhatsApp Chat with Rajesh Kumar.txt`) from Whatsapp in it (as described in the [Readme.md](https://github.com/swahareddy/gpt2AI_whatsapp_simulation/blob/master/README.md))

Then change the add the name of the person who's chat you want to use

In [6]:
chat_name="Rajesh Kumar"
chat_name_formatted=chat_name.lower().replace(" ","_")

### Format Whatsapp Chat
Convert messages like `13/03/20, 11:34 - Tejas: What's the problem` into `Tejas: What's the problem`



In [ ]:
file=open("/content/drive/My Drive/Colab Notebooks/GPT2_WA/Original chats/WhatsApp Chat with "+chat_name+".txt",'r', encoding="utf8")
chat_txt=file.read()
print(chat_txt)
file.close()

import re

lines_match=[]
timestamp_regex="[0-9][0-9]/[0-9][0-9]/[0-9][0-9], [0-9][0-9]:[0-9][0-9] - "
[lines_match.append([m.start(0), m.end(0)]) for m in re.finditer(timestamp_regex, chat_txt)]
# print(lines_match)

from operator import itemgetter
lines_match_sorted=sorted(lines_match, key=itemgetter(0))

chat_wo_ts=""

index_c=0
for name_match in lines_match_sorted:
    if index_c+1<len(lines_match_sorted):
        chat_wo_ts=chat_wo_ts+chat_txt[name_match[1]:lines_match_sorted[index_c+1][0]]
        index_c=index_c+1
    else:
        #print(chat_txt[name_match[1]:])
        chat_wo_ts=chat_wo_ts+chat_txt[name_match[1]:]
        
print(chat_wo_ts)
f = open("/content/drive/My Drive/Colab Notebooks/GPT2_WA/Formatted chats/formatted_"+chat_name_formatted+".txt", "a",  encoding="utf8")
f.write(chat_wo_ts)
f.close()

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [41]:
gpt2.download_gpt2(model_name="774M")

Fetching checkpoint: 1.05Mit [00:00, 441Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 47.8Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 780Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [00:49, 62.4Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 378Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 157Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 151Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [42]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [43]:
file_name = "formatted_"+chat_name_formatted+".txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

In [ ]:
drive_file_path="Colab Notebooks/GPT2_WA/Formatted chats/"+file_name
import os
home_path="/content"
os.chdir(home_path)
import shutil
shutil.copyfile("/content/drive/My Drive/" + drive_file_path, home_path+"/"+file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
run_name_var="run_"+chat_name_formatted
print(run_name_var)

In [ ]:
sess = gpt2.start_tf_sess()
# import tensorflow
# tensorflow.reset_default_graph()
gpt2.finetune(sess,
              dataset=file_name,
              model_name='774M',
              steps=750,
              restore_from='fresh',
              run_name=run_name_var,
              print_every=10,
              sample_every=250,
              save_every=250
              )

After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [57]:
# gpt2.copy_checkpoint_to_gdrive(run_name=run_name_var)

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [8]:
gpt2.copy_checkpoint_from_gdrive(run_name=run_name_var)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [10]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name_var)

'''
Might get error ValueError: Variable model/wpe already exists
In that case, restart runtime and rerun first 2 cells and the run_name_var definition cell
'''

Loading checkpoint checkpoint/run_tripti_rajput/model-750
INFO:tensorflow:Restoring parameters from checkpoint/run_tripti_rajput/model-750


'\nMight get error ValueError: Variable model/wpe already exists\nIn that case, restart runtime and rerun (first 2 cells + run_name_var)\n'

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name=run_name_var)

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              run_name=run_name_var,
              length=1000,
              top_k=40,
              top_p=0.9,
              temperature=1.0,
              nsamples=20,
              batch_size=10
              )

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [17]:
text = gpt2.generate(sess, run_name=run_name_var, return_as_list=True)[0]

In [34]:
gen_file = 'gpt2_gentext_'+chat_name_formatted+'_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      run_name=run_name_var,
                      length=600,
                      temperature=0.9,
                      nsamples=50,
                      batch_size=10
                      )

# files.download("gen_file") #Download to your computer
import shutil
gdrive_destn="/content/drive/My Drive/Colab Notebooks/GPT2_WA/Predicted chats"
shutil.copyfile(gen_file, gdrive_destn+"/"+gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [1]:
model_name = "774M"

gpt2.download_gpt2(model_name=model_name)

In [4]:
sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, model_name=model_name)

Loading pretrained model models/774M/model.ckpt
INFO:tensorflow:Restoring parameters from models/774M/model.ckpt


In [6]:
gpt2.generate(sess,
              model_name=model_name,
              prefix="Studying engineering in India",
              length=100,
              temperature=0.7,
              top_p=0.9,
              nsamples=5,
              batch_size=5
              )

Studying engineering in India, I had the privilege of working with the first ever Indian engineering team to win the World Cup in 2012. But I didn't know at the time that I would be studying engineering at Stanford. I started working at Stanford when I was in high school. I had a lot of fun and I learned a lot. I started studying engineering at Stanford because I liked the way it looked, the way it felt. And that was the first time I really got into the field.

The first semester
Studying engineering in India

The average engineering education in India is a little more than one year long, with the first year being dedicated to basic engineering.

Most engineering courses in India are taught in English, and you are expected to learn the language by the end of the first year.

As for the second year, you are expected to learn the subject in your native language.

You are also expected to complete a minimum of 100 credits.

Some engineering courses in India are offered at the
Studying eng

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

`ValueError: Variable model/wpe already exists, disallowed.`

I would try, in this order:

* to restart the Google Colab session,
* to use a different run_name.

`!kill -9 -1`

# Credits
Originally by [Max Woolf](http://minimaxir.com)
For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read his [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for more information how to use this notebook!

